In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics 8.3.84 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.0/112.6 GB disk)


In [ ]:
import albumentations as A
import numpy as np
import cv2
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import yaml
import os
import random
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ultralytics.utils import metrics

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
# @title Default title text
# from glob import glob
# from sklearn.model_selection import train_test_split

# # Paths to dataset
# dataset_path = "/content/drive/MyDrive/Colab Notebooks/Brain_Tumor/images"
# yes_images = glob(os.path.join(dataset_path, "yes", "*.png"))
# no_images = glob(os.path.join(dataset_path, "no", "*.png"))

# print("Yes images:", len(yes_images))
# print("No images:", len(no_images))

# # Split into training (80%) and validation (20%)
# train_yes, val_yes = train_test_split(yes_images, test_size=0.2, random_state=42)
# train_no, val_no = train_test_split(no_images, test_size=0.2, random_state=42)

# # Output YOLO dataset paths
# output_path = "/content/drive/MyDrive/Colab Notebooks/Brain_Tumor/ready_dataset"

# # Create the required folder structure
# for split in ["train", "validation"]:
#     os.makedirs(f"{output_path}/{split}/images", exist_ok=True)
#     os.makedirs(f"{output_path}/{split}/labels", exist_ok=True)

# # Function to auto-detect tumor (for YES class)
# def detect_tumor(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # If tumor detected, return bounding box
#     if contours:
#         x, y, w, h = cv2.boundingRect(contours[0])
#         return x, y, w, h
#     return None

# # Function to process images and save them in YOLO format
# def process_images(image_list, split, class_id):
#     for img_path in tqdm(image_list, desc=f"Processing {split} - {class_id}"):
#         image = cv2.imread(img_path)
#         h, w, _ = image.shape

#         # Get bounding box for tumors in 'yes' class
#         bbox = detect_tumor(image) if class_id == 0 else None

#         # Convert bbox to YOLO format
#         if bbox:
#             x, y, bw, bh = bbox
#             x_center, y_center = (x + bw/2) / w, (y + bh/2) / h
#             bw, bh = bw / w, bh / h
#             yolo_annotation = f"{class_id} {x_center} {y_center} {bw} {bh}\n"
#         else:
#             yolo_annotation = ""

#         # Save image
#         filename = os.path.basename(img_path)
#         cv2.imwrite(f"{output_path}/{split}/images/{filename}", image)

#         # Save label if tumor is detected
#         if yolo_annotation:
#             label_path = f"{output_path}/{split}/labels/{filename.replace('.png', '.txt')}"
#             with open(label_path, "w") as f:
#                 f.write(yolo_annotation)

# # Process both training and validation sets
# process_images(train_yes, "train", 0)  # Yes Tumor -> Class 0
# process_images(val_yes, "validation", 0)

# process_images(train_no, "train", 1)  # No Tumor -> Class 1
# process_images(val_no, "validation", 1)

# print("Dataset preparation completed successfully!")

In [ ]:
# @title Default title text
# import os
# import cv2
# import numpy as np
# from glob import glob
# from tqdm import tqdm

# # Define dataset paths
# dataset_path = "/content/drive/MyDrive/Colab Notebooks/Brain_Tumor/images"  # Change this to your dataset location
# output_path = "/content/drive/MyDrive/Colab Notebooks/Brain_Tumor/ready_dataset"  # Change this to where you want YOLO dataset

# # Define class IDs
# class_map = {"yes": 0, "no": 1}

# # Create YOLOv8 formatted directories
# for split in ["Train", "Val"]:
#     for cls in class_map.keys():
#         os.makedirs(os.path.join(output_path, split, cls, "images"), exist_ok=True)
#         os.makedirs(os.path.join(output_path, split, cls, "labels"), exist_ok=True)

# # Function to detect tumor (bounding box) in 'Yes' class
# def detect_tumor(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # If tumor detected, return bounding box
#     if contours:
#         x, y, w, h = cv2.boundingRect(contours[0])
#         return x, y, w, h
#     return None

# # Function to process images and generate YOLO labels
# def process_images(class_name, split):
#     image_paths = glob(os.path.join(dataset_path, class_name, "*.png"))  # Change extension if needed

#     for img_path in tqdm(image_paths, desc=f"Processing {split}/{class_name}"):
#         image = cv2.imread(img_path)
#         h, w, _ = image.shape
#         filename = os.path.basename(img_path)

#         # Bounding box for tumors in 'Yes' class
#         if class_name == "yes":
#             bbox = detect_tumor(image)
#         else:
#             bbox = None

#         # Convert bbox to YOLO format
#         if bbox:
#             x, y, bw, bh = bbox
#             x_center, y_center = (x + bw / 2) / w, (y + bh / 2) / h
#             bw, bh = bw / w, bh / h
#             yolo_annotation = f"{class_map[class_name]} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n"
#         else:
#             yolo_annotation = f"{class_map[class_name]} 0.5 0.5 1.0 1.0\n"  # Full image for 'No' class

#         # Save image & label
#         split_dir = os.path.join(output_path, split, class_name)
#         cv2.imwrite(os.path.join(split_dir, "images", filename), image)

#         label_path = os.path.join(split_dir, "labels", filename.replace(".png", ".txt"))
#         with open(label_path, "w") as f:
#             f.write(yolo_annotation)

# # Process Train & Validation sets
# for split in ["Train", "Val"]:
#     for cls in class_map.keys():
#         process_images(cls, split)

# print("✅ Labels generated in YOLOv8 format!")


In [ ]:
import os
import cv2
import numpy as np
import shutil
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define dataset paths
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Brain_Tumor/images"
output_path = "/content/drive/MyDrive/Colab Notebooks/Brain_Tumor/ready_dataset"

# Define class IDs
class_map = {"yes": 0, "no": 1}

# Create YOLOv8 formatted directories
for split in ["Train", "Val"]:
    for cls in class_map.keys():
        os.makedirs(os.path.join(output_path, split, cls, "images"), exist_ok=True)
        os.makedirs(os.path.join(output_path, split, cls, "labels"), exist_ok=True)

# Function to detect tumor (bounding box) in 'Yes' class
def detect_tumor(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If tumor detected, return bounding box
    if contours:
        x, y, w, h = cv2.boundingRect(contours[0])
        return x, y, w, h
    return None

# Get image paths
yes_images = glob(os.path.join(dataset_path, "yes", "*.png"))
no_images = glob(os.path.join(dataset_path, "no", "*.png"))

# Split into Train (80%) & Validation (20%)
yes_train, yes_val = train_test_split(yes_images, test_size=0.2, random_state=42)
no_train, no_val = train_test_split(no_images, test_size=0.2, random_state=42)

# Dictionary to store splits
data_splits = {
    "Train": {"yes": yes_train, "no": no_train},
    "Val": {"yes": yes_val, "no": no_val}
}

# Function to process images and generate YOLO labels
def process_images(class_name, split, images):
    for img_path in tqdm(images, desc=f"Processing {split}/{class_name}"):
        image = cv2.imread(img_path)
        h, w, _ = image.shape
        filename = os.path.basename(img_path)

        # Bounding box for tumors in 'Yes' class
        if class_name == "yes":
            bbox = detect_tumor(image)
        else:
            bbox = None

        # Convert bbox to YOLO format
        if bbox:
            x, y, bw, bh = bbox
            x_center, y_center = (x + bw / 2) / w, (y + bh / 2) / h
            bw, bh = bw / w, bh / h
            yolo_annotation = f"{class_map[class_name]} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n"
        else:
            yolo_annotation = f"{class_map[class_name]} 0.5 0.5 1.0 1.0\n"  # Full image for 'No' class

        # Save image & label
        split_dir = os.path.join(output_path, split, class_name)
        shutil.copy(img_path, os.path.join(split_dir, "images", filename))

        label_path = os.path.join(split_dir, "labels", filename.replace(".png", ".txt"))
        with open(label_path, "w") as f:
            f.write(yolo_annotation)

# Process Train & Validation sets
for split, classes in data_splits.items():
    for cls, images in classes.items():
        process_images(cls, split, images)

print("✅ Dataset split & YOLO labels generated successfully!")


Processing Val/no: 100%|██████████| 1910/1910 [26:57<00:00,  1.18it/s]

✅ Dataset split & YOLO labels generated successfully!
